In [1]:
%run 'ks25_Read_epistasis.ipynb'

/home/katya/anaconda3/envs/python2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


ImportError: No module named pymol

11510
259

Reading precalculated epistasis...
/home/katya/start/HIS3InterspeciesEpistasis/Analysis/Karen/files_dump/epistasis_in_foursomes/



Variables: 

- ep_dict: a dictionary, Keys are tuples (mutA, mutB) and values are pandas.Series of epistasis between mutA and mutB in all backgrounds. 
Tuples (mutA, mutB) are sorted by aa_position.

- get_ep_summary: a function that takes ep_dict as an argument and applies an arbitrary function to every pd.Series object. For example, ep_summary = get_ep_summary(ep_dict, np.mean)

- get_ep_summary_by_positions: a function that uses get_ep_summary and then averages results by position_pair. For example, get_ep_summary_by_positions(ep_dict, np.max, np.mean) will apply np.max to epistasis series and then use np.mean to average across a position pair. 

Important: natlib_filtering is currently True. This can be changed in the cell #2.



In [52]:
# getting segment numbers for every position
position_to_segment = {}
for segment in positions.segment:
    for position in positions.set_index('segment').loc[segment]['positions_Uniprot_P06633']:
        position_to_segment[position] = segment

In [63]:
def get_strong_epistasis_percents(series_of_impacts, threshold=0.4):
    negative = series_of_impacts[series_of_impacts < -1 * threshold]
    positive = series_of_impacts[series_of_impacts > threshold]
    neutral = series_of_impacts[(series_of_impacts <= threshold) & (series_of_impacts >= -1 * threshold)]
    fraction_negative = 1. * len(negative) / len(series_of_impacts)
    fraction_positive = 1. * len(positive) / len(series_of_impacts)
    fraction_neutral = 1. * len(neutral) / len(series_of_impacts)
    return fraction_negative, fraction_neutral, fraction_positive

def get_epistatic_backgrounds(segment, mutA, mutB, backgrounds):
    mutAB_full_mutations = [get_AB(bg, mutA, mutB) for bg in backgrounds]
    mutAB_aaseq_list = []
    for m in mutAB_full_mutations:
        try:
            if 'scaled' not in segment: segment += '_scaled'
            aa_seq = find_genotype(data_natural[segment], m).aa_seq.values[0]
            mutAB_aaseq_list.append(aa_seq)
        except AttributeError:
            pass
    return mutAB_aaseq_list

In [58]:
ep_summary = get_ep_summary(ep_dict, get_strong_epistasis_percents)
sorted_ep_summary = sorted(ep_summary, key=lambda k: max([ep_summary[k][0], ep_summary[k][2]]), reverse=True)

In [65]:
strong_epistasis_threshold = 0.4
ratio_threshold = 5
threshold = 0.4
sample_size = 10
funny_genotypes = OrderedDict()
counter = 0
for mutA, mutB in sorted_ep_summary:
    segment = position_to_segment[int(mutA[:-1])]
    fractions = {ep_summary[mutA, mutB][0] : -1, ep_summary[mutA, mutB][2] : 1}
    major_peak = max(ep_summary[mutA, mutB][0], ep_summary[mutA, mutB][2])
    minor_peak = min(ep_summary[mutA, mutB][0], ep_summary[mutA, mutB][2])
    if minor_peak > 0:
        if major_peak / minor_peak > ratio_threshold:
            filtered = ep_dict[mutA, mutB][ep_dict[mutA, mutB] * fractions[minor_peak] > strong_epistasis_threshold]
            if len(filtered) > sample_size:
                minor_genotypes = filtered.sample(sample_size).index.values
    
            funny_genotypes[segment] = get_epistatic_backgrounds(segment, mutA, mutB, minor_genotypes)
            print funny_genotypes
    counter += 1
    if counter == 40:
        break

OrderedDict([('S2', ['SNRPYCFTDLGLQREKVGDLSTEMIPHV', 'SGRPYCFTKLGLQREKVGDLSSEMIPHV', 'SGRAYAVTKLGLQREKVGDLSSEMVTHV', 'SDRPYAFTDLGLQREKVGDLSTEMIPHV', 'SSRPFCFTNLGLQREKVGDLSTEMIPHV', 'SGRPFCVTDLGLQREKVGDLSCEMIPHV', 'SGRPYAVTELGLQREKVGDLSTEMIPHV', 'SGRAFCVTDLGLQREKVGDLSSEMVTHV', 'SNRPYCVTELGLQREKVGDLSSEMIPHV'])])
OrderedDict([('S2', ['SNRPYCFTDLGLQREKVGDLSTEMIPHV', 'SGRPYCFTKLGLQREKVGDLSSEMIPHV', 'SGRAYAVTKLGLQREKVGDLSSEMVTHV', 'SDRPYAFTDLGLQREKVGDLSTEMIPHV', 'SSRPFCFTNLGLQREKVGDLSTEMIPHV', 'SGRPFCVTDLGLQREKVGDLSCEMIPHV', 'SGRPYAVTELGLQREKVGDLSTEMIPHV', 'SGRAFCVTDLGLQREKVGDLSSEMVTHV', 'SNRPYCVTELGLQREKVGDLSSEMIPHV']), ('S1', ['KAVGQIKGLERFGSGFAPLDEALSRSVVDL', 'EAVGDVTGIERFGSGFAPLDEALSRSVVDL', 'DALGEVRGIERFGSGFAPLDEALSRSVVDF', 'KAVGEFTGLERFGSGFAPLDEALSRSVVDL', 'KAMGDVTGVERFGSGFAPLDEALSRSVVDL', 'EALGEVKGIERFGSGFAPLDEALSRSVVDL', 'DALGDVKGLERFGSGFAPLDEALSRSVVDL', 'EAMGQIKGVERFGSGFAPLDEALSRSVVDL', 'EAVGQFTGLERFGSGFAPLDEALSRSVVDL', 'DAMGPIKGVERFGSGFAPLDEALSRSVVDL'])])
OrderedDict([('S2', ['SGRA

In [15]:
ep_dict[('143T', '163V')].sort_values()[(ep_dict[('143T', '163V')] > -1) & (ep_dict[('143T', '163V')] < -0.6)].sample(5)

wt
139A:141S:144K:160V:161T        -0.608742
139A:141G:144N:157T             -0.627598
137S:140F:141G:144N:157S:160V   -0.863444
137S:140F:142F:144N:157S:161T   -0.805598
141S:142F:144D:157S             -0.750049
Name: mut_list_Scer, dtype: float64

In [23]:
sq = get_AB('137S:140F:141G:144N:157S:160V', '143T', '163V')

In [25]:
find_genotype(data_natural['S2_scaled'], sq).aa_seq.values[0]

AttributeError: 'NoneType' object has no attribute 'aa_seq'